In [29]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,rank
from pyspark.sql.window import Window

In [30]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import min

# Initialize SparkSession
spark = SparkSession.builder.appName("Create DataFrame Example").getOrCreate()

# Define the data
data = [
    (1, 2, '2016-03-01', 5),
    (1, 2, '2016-05-02', 6),
    (2, 3, '2017-06-25', 1),
    (3, 1, '2016-03-02', 0),
    (3, 4, '2018-07-03', 5)
]

# Define the schema
columns = ["player_id", "device_id", "event_date", "games_played"]

# Create DataFrame
activity_df = spark.createDataFrame(data, schema=columns)

# Show the DataFrame
activity_df.show()


+---------+---------+----------+------------+
|player_id|device_id|event_date|games_played|
+---------+---------+----------+------------+
|        1|        2|2016-03-01|           5|
|        1|        2|2016-05-02|           6|
|        2|        3|2017-06-25|           1|
|        3|        1|2016-03-02|           0|
|        3|        4|2018-07-03|           5|
+---------+---------+----------+------------+



In [31]:
WindowSpec = Window.partitionBy("player_id").orderBy("event_date")
rank_df = activity_df.withColumn("rank_device",rank().over(WindowSpec))
rank_df.show()



+---------+---------+----------+------------+-----------+
|player_id|device_id|event_date|games_played|rank_device|
+---------+---------+----------+------------+-----------+
|        1|        2|2016-03-01|           5|          1|
|        1|        2|2016-05-02|           6|          2|
|        2|        3|2017-06-25|           1|          1|
|        3|        1|2016-03-02|           0|          1|
|        3|        4|2018-07-03|           5|          2|
+---------+---------+----------+------------+-----------+



In [34]:
#to report the device that is first logged in for each player.
result_df = rank_df.select("player_id","device_id").filter(rank_df.rank_device == 1)
result_df.show()

+---------+---------+
|player_id|device_id|
+---------+---------+
|        1|        2|
|        2|        3|
|        3|        1|
+---------+---------+

